In [ ]:
import csv, json, math, os
from osgeo import ogr
from random import uniform
from shapely.geometry import shape, Point, Polygon

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def RandomPointFromBbox(bbox):
    l,b,r,t = bbox
    point = Point(uniform(l,r),uniform(t,b))
    return point.__geo_interface__['coordinates']

def GetPointsFromData(data, minPointCount = 100.0):
    points = []
    pointCount = max(float(minPointCount), len(data["features"]))
    geom = data["features"][0]["geometry"]
    polygon = shape(geom)
    bbox = polygon.bounds
    for i in range(minPointCount):
        points.append(RandomPointFromBbox(bbox))
    return points


In [ ]:
with open("list.json") as f:
    countryList = json.load(f)
    
with open("centroids.geojson") as f:
    centroids = json.load(f)

In [ ]:
for country in countryList:
    match = False
    for feature in centroids["features"]:
        for name in feature["properties"]["names"]:
            if country['name_en'] == name:
                match = True
                break
        if match:
            break
    if not match:
        print "Missing %s" % country["name_en"]

In [ ]:
unhcrData = []
with open("unhcr_popstats_export_time_series_all_data.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        unhcrData.append(row)

In [ ]:
refugees = {}
for row in unhcrData:
    year = row['Year']
    value = row['Value']
    org = row['Origin']
    dst = row['Country / territory of asylum/residence']
    if int(year) >= 1983:
        if row['Population type'] == 'Refugees (incl. refugee-like situations)':
            if org != 'Various/Unknown' and\
                dst != 'Various/Unknown' and\
                org != 'Stateless' and\
                dst != "Stateless":
                if org not in refugees:
                    refugees[org] = {dst: {'max': 0}}
                if dst not in refugees[org]:
                    refugees[org][dst] = {'max': 0}
                refugees[org][dst][year] = value if value != "*" else "0"


In [ ]:
countryNames = []
for org in refugees:
    countryNames.append(org)
    for dst in refugees[org]:
        countryNames.append(dst)
countryNames = list(set(countryNames))

In [ ]:
missing = []
countryName2Iso = {}
for country in countryNames:
    match = False
    for feature in centroids["features"]:
        for name in feature["properties"]["names"]:
            if unicode(country,"utf8").encode('ascii','ignore') == name:
                countryName2Iso[country] = feature["properties"]["iso_alpha-3"]
                match = True
                break
        if match:
            break
    if not match:
        print "Missing %s" % country
        missing.append(country)

In [ ]:
for org in refugees:
    for dst in refugees[org]:
        values = refugees[org][dst]
        for year in range(1984,2017):
            prevValue = int(values[str(year-1)]) if str(year-1) in values else 0
            currValue = int(values[str(year)]) if str(year) in values else 0
            if currValue - prevValue > values['max']:
                values['max'] = currValue - prevValue
            

In [155]:
randomPoints = {}
for name in countryName2Iso:
    iso = countryName2Iso[name]
    filename = "gadm28_levels/%s/%s_adm0.geojson" % (iso,iso)
    if os.path.exists(filename):
        with open(filename) as f:
            geojson = json.load(f)
            points = GetPointsFromData(geojson, 1000)            
            randomPoints[iso] = points
    else:
        print "Missing %s" % filename
        l,b,r,t = (66, 23, 108, 42)
        points = []
        for i in range(1000):
            point = Point(uniform(l,r),uniform(t,b))
            points.append(point.__geo_interface__['coordinates'])
        randomPoints[iso] = points

Missing gadm28_levels/XTX/XTX_adm0.geojson


In [35]:
iso2Centroids = {}
for name in countryName2Iso:
    iso = countryName2Iso[name]
    match = False
    for feature in centroids["features"]:
        if feature["properties"]["iso_alpha-3"] == iso:
            match = True
            break
    iso2Centroids[iso] = feature["geometry"]["coordinates"]


In [38]:
LonLatToPixelXY(iso2Centroids['AFG'])

[174.93333333333334, 103.11672116189807]

In [52]:
LonLatToPixelXY(randomPoints['AFG'][0])

[175.13931132111608, 103.79717529928455]

In [171]:
def xy2rgba(coords):
    x,y = coords
    _max = 1.0;
    _min = 0.0
    r = math.floor(255.0 * (x - _min) / (_max - _min));
    g = math.floor(255.0 * (y - _min) / (_max - _min));
    b = 0.0;
    a = 255.0;
    return [r,g,b,a]

def rgba2xy(rgba):
    return [rgba[0] / 255.0 + rgba[2], (rgba[1] + rgba[3]/255.0)/ 255.0]

In [187]:
# Uniqueness test
t = 0
for keys in randomPoints:
    test = []
    for point in randomPoints[keys]:
        a = LonLatToPixelXY(point)
        test.append(xy2rgba((a[0]/255.,a[1]/255.)))
    
    unique_data = [list(x) for x in set(tuple(x) for x in test)]
    if len(unique_data) < 100:
        print keys, len(unique_data)
        t += 1
print t

LIE 1
EGY 80
BGD 20
QAT 3
BGR 23
GHA 20
CPV 9
HND 24
ROU 47
JOR 19
LBR 16
VNM 72
PRI 6
SXM 1
PRK 30
TZA 79
KHM 20
NIC 20
TTO 4
PRY 50
HKG 2
LBN 9
SVN 6
BFA 30
SVK 17
HRV 25
KNA 1
JAM 4
GIB 1
DJI 6
GIN 30
URY 25
VAT 1
SYC 50
NPL 28
MAR 79
YEM 59
PHL 96
GAB 25
SYR 30
MAC 1
BRB 1
COK 69
TCA 4
NIU 1
DMA 2
BEN 15
NGA 70
BEL 12
DEU 67
LKA 11
MWI 21
CRI 24
CMR 54
LSO 4
UGA 24
TKM 88
SUR 16
NLD 18
COM 2
BMU 2
GEO 18
MNE 6
MHL 72
MTQ 1
BLZ 6
BDI 6
AFG 98
NFK 1
VGB 2
BLR 56
GRD 1
GRC 55
ZWE 41
RWA 6
TJK 34
THA 76
HTI 6
PSE 4
LCA 2
VCT 1
BTN 6
MYS 89
CZE 24
ATG 2
MUS 48
DOM 12
LUX 2
ISR 10
SMR 1
VUT 24
GNQ 24
COG 47
GLP 1
GUF 12
TLS 6
BWA 56
MDA 16
MDG 66
ECU 76
SEN 25
ESH 42
MDV 14
ASM 12
SPM 4
SRB 20
AND 1
LTU 24
ZMB 74
WLF 4
SSD 63
IRL 24
GTM 16
DNK 30
AZE 20
AUT 28
PLW 12
KEN 48
LAO 43
TUR 94
ALB 8
OMN 52
TUV 12
BRN 4
TUN 28
HUN 24
CUW 2
MKD 9
WSM 4
GNB 9
SWZ 6
TON 18
CIV 30
KOR 31
AIA 2
GUY 28
CHE 12
CYP 3
BIH 12
SGP 4
SOM 87
CAF 70
POL 54
KWT 4
GMB 3
ERI 34
TGO 12
CYM 4
EST 24
LVA 29
IRQ 63

In [161]:
# Encode web mercator floats into rgba
def frac(x):
    return x - math.floor(x)

def float2rgba(value):
    x,y = math.floor(value), frac(value)
    x_max = 255.0
    x_min = 0.0
    y_min = 0.0
    y_max = 1.0
    r = math.floor(255.0 * (x - x_min) / (x_max - x_min));
    g = math.floor(255.0 * (y - y_min) / (y_max - y_min));
    b = 0.0;
    a = 255.0;
    return [r,g,b,a]

def rgba2float(rgba):
    value = [rgba[0] / 255.0 + rgba[2], (rgba[1] + rgba[3]/255.0)/ 255.0]
    return value[0] + value[1]


In [178]:
t = 0
for keys in randomPoints:
    test2 = []
    for point in randomPoints[keys]:
        a = LonLatToPixelXY(point)
        test2.append(float2rgba(a[0]))    
    unique_data = [list(x) for x in set(tuple(x) for x in test2)]
    if len(unique_data) < 100:
        print keys, len(unique_data)
        t +=1
print t

LIE 31
SXM 27
KNA 60
GIB 6
VAT 4
MAC 14
BRB 43
NIU 33
DMA 45
BMU 45
MTQ 77
NFK 16
GRD 78
LCA 39
VCT 63
SMR 22
SPM 52
AND 68
CUW 96
AIA 92
SGP 88
MLT 73
ABW 37
MCO 7
NRU 10
25
